In [1]:
pip install tensorflow scikit-learn numpy


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl (2.1 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical


In [5]:
def preprocess_data(directory, img_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='training')

    validation_generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation')
    
    return train_generator, validation_generator

train_data, val_data = preprocess_data('dataset-1')


Found 3282 images belonging to 26 classes.
Found 810 images belonging to 26 classes.


In [6]:
def create_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

vgg16 = VGG16(weights='imagenet', include_top=False)
inception = InceptionV3(weights='imagenet', include_top=False)
resnet = ResNet50(weights='imagenet', include_top=False)

models = [create_model(vgg16, train_data.num_classes),
          create_model(inception, train_data.num_classes),
          create_model(resnet, train_data.num_classes)]




94765736/94765736 [==============================] - 26s 0us/step


In [7]:
for model in models:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_data, validation_data=val_data, epochs=10)



Epoch 1/10


103/103 [==============================] - 1120s 11s/step - loss: 3.5879 - accuracy: 0.0753 - val_loss: 3.0470 - val_accuracy: 0.0790
Epoch 2/10
103/103 [==============================] - 1100s 11s/step - loss: 3.0547 - accuracy: 0.0728 - val_loss: 3.0583 - val_accuracy: 0.0864
Epoch 3/10
103/103 [==============================] - 1110s 11s/step - loss: 3.0632 - accuracy: 0.0783 - val_loss: 3.0551 - val_accuracy: 0.0840
Epoch 4/10
103/103 [==============================] - 1210s 12s/step - loss: 3.0573 - accuracy: 0.0820 - val_loss: 3.0486 - val_accuracy: 0.0765
Epoch 5/10
103/103 [==============================] - 1226s 12s/step - loss: 3.0424 - accuracy: 0.0884 - val_loss: 2.9964 - val_accuracy: 0.1284
Epoch 6/10
 71/103 [===================>..........] - ETA: 5:57 - loss: 2.9753 - accuracy: 0.1196

KeyboardInterrupt: 